In [14]:
!python -m spacy download it_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')


In [16]:
pip install pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.3-py3-none-any.whl size=16463 sha256=a216eae06242058761247841f077f89f3bf5ee80dbb3665e08f3c79b86be1dfb
  Stored in directory: /root/.cache/pip/wheels/3c/a4/19/02a1f08d032a017d5d7e22da595aa652ba0a2f2e22de73981b
Successfully built pandarallel


In [17]:
#getting data into dataframe
!python /itdi/generation.py

Generating the dataset...
100% 11/11 [00:02<00:00,  4.20it/s]
Cleaning text...
Splitting sentences...
/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value ['ner', 'lemmatizer', 'textcat', 'custom', 'tagger'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],
   0.00%                                          |        0 /   109444 
   0.00%                                          |        0 /   10944M   0.00%                                          |        0 /   109444 
   1.00%                                          |     1094 /   10944M   0.00%                                          |        0 /   109444 
   1.05%                                          |     1152 /   10944M   0.00%                                          |        0 /   109444 
   1.14%                                          |     1245 /   1

In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('train.csv')

In [13]:
df.head()

,text,label
0,Ète 'u presidende d'a Commissione europèe de ô...,7
1,Barroso s'ète laureate jndr'à giurisprudenze j...,7
2,Jndre quèste urteme universetate hé cchiù nnan...,7
3,Hé fatte pure alcune periode de studie e stage...,7
4,"Barroso ète spusate e tène ttré figghie, Luís,...",7


In [14]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification

In [16]:
model = "dbmdz/bert-base-italian-cased"
tokenizer = BertTokenizer.from_pretrained(model)

In [17]:
token_id = []
attention_masks = []

def pre_processing(input_text, tokenizer):
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 32,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )

In [18]:
from tqdm import tqdm
for i in tqdm(df['text'].values):
  encoded_text = pre_processing(i,tokenizer)
  token_id.append(encoded_text['input_ids'])
  attention_masks.append(encoded_text['attention_mask'])

  0%|          | 0/383980 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 383980/383980 [03:57<00:00, 1615.36it/s]


In [19]:
dev_labels = []
dev_text = []
f = open('/itdi/data/dev.txt','r')
for i in f:
  dev_labels.append(i.split()[0])
  dev_text.append(' '.join(i.split()[1:]))

In [20]:
df_dev = pd.DataFrame({'text':dev_text,'label':dev_labels},columns=['text','label'])

In [21]:
df_dev

,text,label
0,"Ma te see foeura de coo, ò te me toeuvet in gir?.",LMO
1,a carni re puvireddi.,SCN
2,L'era on simpatich tradizional appontament con...,LMO
3,Chi un tenpo xè stà zovane adesso se consola b...,VEC
4,"ma òua scià no coménse a fâse do mâ da sôlo, a...",LIJ
...,...,...
6794,"Atorno de ela se ghe taca tante altre anime, e...",VEC
6795,Stu disgrazziatu 'un mi voli maritari picchì d...,SCN
6796,L'urtima vota mi parsi ca aveva 'nt' 'a manu u...,SCN
6797,"Chi è malatu, 'u figghiozzu?",SCN


In [22]:
fold_label = {
    'EML' : 0,
    'NAP' : 1,
    'PMS' : 2,
    'FUR' : 3,
    'LLD' : 4,
    'LIJ' : 5,
    'LMO' : 6,
    'ROA_TARA' : 7,
    'SCN' : 8,
    'VEC' : 9,
    'SC' : 10
}

df_dev['label'] = [fold_label[i] for i in df_dev['label'].values]

In [23]:
token_id_ = []
attention_masks_ = []

In [24]:
for i in tqdm(df_dev['text'].values):
  encoded_text_ = pre_processing(i,tokenizer)
  token_id_.append(encoded_text_['input_ids'])
  attention_masks_.append(encoded_text_['attention_mask'])

  0%|          | 0/6799 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 6799/6799 [00:03<00:00, 1937.56it/s]


In [25]:
labels = df.label
labels_ = df_dev.label

In [26]:
# converting train data to tensor
token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)
# converting validation data to tensor
token_id_ = torch.cat(token_id_, dim = 0)
attention_masks_ = torch.cat(attention_masks_, dim = 0)
labels_ = torch.tensor(labels_)

In [27]:
train_set = TensorDataset(token_id,attention_masks,labels)
val_set = TensorDataset(token_id_,attention_masks_,labels_)

In [28]:
batch_size=1024
train_dataloader = DataLoader(train_set,sampler=RandomSampler(train_set),batch_size=batch_size) 
val_dataloader = DataLoader(val_set,sampler=SequentialSampler(val_set),batch_size=batch_size)

In [29]:
clf = BertForSequenceClassification.from_pretrained(
    model,
    num_labels = 11,
    output_attentions = False,
    output_hidden_states = False,
)

Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-italian-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model c

In [30]:
import numpy as np
optimizer = torch.optim.AdamW(clf.parameters(),lr = 5e-5,eps = 1e-08)

# Run on GPU
clf.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31102, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [31]:
def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity


In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 2
from tqdm import trange
for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    clf.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = clf(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    clf.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in val_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = clf(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')


Epoch:  50%|█████     | 1/2 [07:13<07:13, 433.94s/it]


	 - Train loss: 0.2506
	 - Validation Accuracy: 0.0000
	 - Validation Precision: 0.0000
	 - Validation Recall: 0.0000
	 - Validation Specificity: 0.0000



Epoch: 100%|██████████| 2/2 [14:25<00:00, 432.69s/it]


	 - Train loss: 0.0626
	 - Validation Accuracy: 0.0000
	 - Validation Precision: 0.0000
	 - Validation Recall: NaN
	 - Validation Specificity: 0.0000



In [33]:
#testing on gold dataset from github repository https://github.com/noe-eva/ITDI_2022
test_labels = []
test_text = []
f = open('/content/test_gold_standard.txt','r')
for i in f:
  test_labels.append(i.split()[0])
  test_text.append(' '.join(i.split()[1:]))

In [34]:
df_test = pd.DataFrame({'text':test_text,'label':test_labels},columns=['text','label'])

In [35]:
df_test

,text,label
0,"E lì è montagnie, là o' se trova asai falchoni...",VEC
1,"Ma alora l é proprio un zoo, l à 'sontà l Pirata.",LLD
2,Al è ancje un aspiet faunistic di segnalâ.,FUR
3,Ca sempe chesto é stato!,NAP
4,Ancje achì al à lis sôs sodisfazions:,FUR
...,...,...
11082,Varda ti sta scioraria:,LLD
11083,Anche da là podarae vede canche rua i nemighe ...,LLD
11084,Al posto sò s à scentà na veceta con un papaga...,LLD
11085,Dû mîṡ fà ai é suzès al diṡâster:,EML


In [37]:
fold_label = {
    'EML' : 0,
    'NAP' : 1,
    'PMS' : 2,
    'FUR' : 3,
    'LLD' : 4,
    'LIJ' : 5,
    'LMO' : 6,
    'ROA_TARA' : 7,
    'SCN' : 8,
    'VEC' : 9,
    'SC' : 10
}

df_test['label'] = [fold_label[i] for i in df_test['label'].values]

In [55]:
def predictor(text,classifier=clf,tokenizer=tokenizer):
  inputs = tokenizer(text,return_tensors='pt')
  inputs.to(device)
  with torch.no_grad():
    logits = clf(**inputs).logits
  predicted_class_id = logits.argmax().item()
  return predicted_class_id

In [56]:
predictor(df_test['text'].values[0],clf,tokenizer)

9

In [57]:
df_test['predicted'] = df_test['text'].apply(predictor)

In [58]:
df_test

,text,label,predicted
0,"E lì è montagnie, là o' se trova asai falchoni...",9,9
1,"Ma alora l é proprio un zoo, l à 'sontà l Pirata.",4,4
2,Al è ancje un aspiet faunistic di segnalâ.,3,3
3,Ca sempe chesto é stato!,1,1
4,Ancje achì al à lis sôs sodisfazions:,3,3
...,...,...,...
11082,Varda ti sta scioraria:,4,8
11083,Anche da là podarae vede canche rua i nemighe ...,4,1
11084,Al posto sò s à scentà na veceta con un papaga...,4,9
11085,Dû mîṡ fà ai é suzès al diṡâster:,0,6


In [59]:
from sklearn.metrics import classification_report

In [60]:
print(classification_report(df_test['label'].values,df_test['predicted'].values))

              precision    recall  f1-score   support

           0       0.99      0.71      0.83       825
           1       0.85      0.77      0.81      2026
           2       0.00      0.00      0.00         0
           3       0.98      0.95      0.96      1323
           4       0.98      0.56      0.71      2200
           5       0.87      0.89      0.88      2282
           6       0.39      0.98      0.56       689
           7       0.57      0.09      0.15       603
           8       0.00      0.00      0.00         0
           9       0.53      0.59      0.56      1139
          10       0.00      0.00      0.00         0

    accuracy                           0.73     11087
   macro avg       0.56      0.50      0.50     11087
weighted avg       0.83      0.73      0.75     11087



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
